## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
0,0,Port Elizabeth,ZA,-33.9180,25.5701,62.89,77,0,3.44,clear sky
1,1,Tubmanburg,LR,6.8708,-10.8172,73.11,91,100,1.52,overcast clouds
2,2,Kodiak,US,57.7900,-152.4072,44.11,76,75,12.66,broken clouds
3,3,Hobart,AU,-42.8794,147.3294,59.25,57,20,8.05,few clouds
4,4,Itarema,BR,-2.9248,-39.9167,80.08,77,3,15.19,clear sky


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("How cold is too cold on your trip? "))
max_temp = float(input("How hot is too hot on your trip? "))

How cold is too cold on your trip? 70
How hot is too hot on your trip? 75


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
1,1,Tubmanburg,LR,6.8708,-10.8172,73.11,91,100,1.52,overcast clouds
9,9,Pemangkat,ID,1.1667,108.9667,74.70,91,94,6.31,overcast clouds
14,14,San Cristobal,VE,7.7669,-72.2250,71.78,78,89,3.24,overcast clouds
21,21,Pisco,PE,-13.7000,-76.2167,72.01,83,40,9.22,mist
28,28,Sao Filipe,CV,14.8961,-24.4956,70.63,71,70,12.01,broken clouds
82,82,Pangkalanbuun,ID,-2.6833,111.6167,72.93,97,100,2.73,overcast clouds
94,94,Banganapalle,IN,15.3167,78.2333,73.99,58,17,2.42,few clouds
108,108,Dokri,PK,27.3743,68.0967,70.77,34,7,10.04,clear sky
115,115,Wattegama,LK,6.7989,81.4808,70.57,95,1,1.83,clear sky
131,131,Gbarnga,LR,6.9956,-9.4722,72.64,98,78,3.67,light rain


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID         74
City            74
Country         74
Lat             74
Lng             74
Max Temp        74
Humidity        74
Cloudiness      74
Wind Speed      74
Weather Desc    74
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Desc", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Desc,Lat,Lng,Hotel Name
1,Tubmanburg,LR,73.11,overcast clouds,6.8708,-10.8172,
9,Pemangkat,ID,74.70,overcast clouds,1.1667,108.9667,
14,San Cristobal,VE,71.78,overcast clouds,7.7669,-72.2250,
21,Pisco,PE,72.01,mist,-13.7000,-76.2167,
28,Sao Filipe,CV,70.63,broken clouds,14.8961,-24.4956,
82,Pangkalanbuun,ID,72.93,overcast clouds,-2.6833,111.6167,
94,Banganapalle,IN,73.99,few clouds,15.3167,78.2333,
108,Dokri,PK,70.77,clear sky,27.3743,68.0967,
115,Wattegama,LK,70.57,clear sky,6.7989,81.4808,
131,Gbarnga,LR,72.64,light rain,6.9956,-9.4722,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    #add to params
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][1]["name"]
    except (IndexError):
        print("Hotel not found....skipping.")
        

Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Weather Desc}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</d1>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))